In [1]:
%matplotlib inline

# Tutorial de redes neuronales

Las redes neuronales se pueden construir usando el paquete `torch.nn`.

Ahora que ha visto `autograd`, `nn` depende `autograd` para definir modelos y diferenciarlos. `nn.Module` contiene capas y un método `forward(input)` que devuelve `output`.

Por ejemplo, mire esta red que clasifica imágenes de dígitos:

![mnist](https://pytorch.org/tutorials/_images/mnist.png)

Es una red feed-forward simple. Toma la entrada, la pasa a través de varias capas una tras otra, y finalmente da la salida.

Un procedimiento de entrenamiento típico para una red neuronal es el siguiente:

 * Definir la red neuronal que tiene algunos parámetros (o pesos) que se pueden aprender
 * Iterar sobre un conjunto de datos de entradas
 * Procesar los datos a través de la red
 * Calcule la pérdida (qué tan lejos está la salida de ser correcta)
 * Propagar gradientes de regreso a los parámetros de la red.
 * Actualice los pesos de la red, normalmente usando una regla de actualización simple: `weight = weight - learning_rate * gradient`

It is a simple feed-forward network. It takes the input, feeds it
through several layers one after the other, and then finally gives the
output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or
  weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``

## Definición de la red

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # kernel
        # 1 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input image channel, 16 output channels, 5x5 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Solo tiene que definir la función `forward`, y la función `backward` (donde se calculan los gradientes) se define automáticamente para usted por parte de `autograd`. Puedes usar cualquiera de las operaciones de Tensor en la función `forward`.

Los parámetros que se pueden aprender de un modelo son devueltos por `net.parameters()`

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Probemos con una entrada aleatoria de 32x32.

 > **Nota**: el tamaño de entrada esperado de esta red (LeNet) es 32x32. Para usar esta red en el conjunto de datos MNIST, cambie el tamaño de las imágenes del conjunto de datos a 32x32.

In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0720, -0.2085, -0.0294, -0.0226,  0.0114, -0.0632,  0.0166, -0.1427,
          0.0070, -0.0554]], grad_fn=<AddmmBackward>)


Ponga a cero los búferes de gradiente de todos los parámetros y backprops con gradientes aleatorios

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

 > **Nota**:
 > `torch.nn` solo admite mini lotes. El paquete `torch.nn` completo solo admite entradas que son un mini lote de muestras, y no una sola muestra.
 >
 > Por ejemplo, `nn.Conv2d` tomará un tensor 4D de `nSamples x nChannels x Height x Width`
 >
 > Si tiene una sola muestra, use `input.unsqueeze(0)` para agregar una dimensión de lote falsa.

Antes de continuar, recapitulemos todas las clases que ha visto hasta ahora.

**Resumen**:
 * `torch.Tensor`- Una *matriz multidimensional* con soporte para operaciones de autograd como `backward()`. También *contiene el tensor* con el gradiente.
 * `nn.Module`- Módulo de red neuronal. *Una forma conveniente de encapsular parámetros*, con helpers para moverlos a la GPU, exportarlos, cargarlos, etc.
 * `nn.Parameter`- Una especie de tensor, que se *registra automáticamente como parámetro cuando se asigna como atributo a* `Module`.
 * `autograd.Function`- Implementa *definiciones hacia adelante y hacia atrás de una operación de autograd*. Cada operación `Tensor` crea al menos un único nodo `Function` que se conecta a las funciones que crearon `Tensor` y *codifica su historial* .

**En este punto, cubrimos**:
 * Definición de una red neuronal
 * Procesando entradas y llamando a backward


**Aún queda**:
 * Calcular la pérdida
 * Actualización de los pesos de la red

## Función de pérdida

Una función de pérdida toma el par de entradas (salida, objetivo) y calcula un valor que estima qué tan lejos está la salida del objetivo.

Hay varias [funciones de pérdida](https://pytorch.org/docs/nn.html#loss-functions) diferentes en el paquete `nn`. Una pérdida simple es: `nn.MSELoss` que calcula el error cuadrático medio entre la entrada y el objetivo.

In [12]:
output = net(input)
print(output.shape)
target = torch.randn(10)  # a dummy target, for example
print(target.shape)
target = target.view(1, -1)  # make it the same shape as output
print(target.shape)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

torch.Size([1, 10])
torch.Size([10])
torch.Size([1, 10])
tensor(1.1937, grad_fn=<MseLossBackward>)


Ahora, si sigue `loss` en la dirección hacia atrás, usando su atributo `.grad_fn`, verá un gráfico de cálculos que se ve así:

```
    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> flatten -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss
```

Entonces, cuando llamamos `loss.backward()`, todo el gráfico se diferencia con los parámetros de la red neuronal, y todos los Tensores en el gráfico que tengan `requires_grad=True` tendrán su `.gradTensor` acumulado con el gradiente.

A modo de ilustración, sigamos algunos pasos hacia atrás:

In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop

Para propagar el error hacia atrás, todo lo que tenemos que hacer es `loss.backward()`. Sin embargo, debe borrar los degradados existentes; de lo contrario, los degradados se acumularán en los degradados existentes.

Ahora llamaremos `loss.backward()` y veremos los gradientes de sesgo de conv1 antes y después del retroceso.

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0174, -0.0032,  0.0136, -0.0195, -0.0014, -0.0151])


Ahora, hemos visto cómo usar las funciones de pérdida.

**Leer más tarde**:

El paquete de red neuronal contiene varios módulos y funciones de pérdida que forman los componentes básicos de las redes neuronales profundas. Una lista completa con documentación está [aquí](https://pytorch.org/docs/nn).


**Lo único que queda por aprender es**:

Actualización de los pesos de la red

## Actualización de los pesos

La regla de actualización más simple utilizada en la práctica es el Descenso de gradiente estocástico (SGD):

`weight = weight - learning_rate * gradient`

Podemos implementar esto usando un código Python simple:

```
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```

Sin embargo, como usa redes neuronales, desea usar varias reglas de actualización diferentes como SGD, Nesterov-SGD, Adam, RMSProp, etc. Para habilitar esto, creamos un pequeño paquete: `torch.optim` que implementa todos estos métodos. Usarlo es muy simple:

In [9]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

 > **Nota**:
 > Observe cómo los búferes de gradiente tuvieron que configurarse manualmente a cero usando `optimizer.zero_grad()`. Esto se debe a que los gradientes se acumulan como se explica en la sección Backprop.